# Gene Therapy Sequence Generator

## Introduction

Welcome to this exciting citizen science project, an initiative aimed at democratizing and promoting open-source research in the field of genomics. This project is a labor of love designed by a wet lab biologist who, with the invaluable assistance of the artificial intelligence model ChatGPT, embarked on a journey to explore the applications of machine learning in the world of genetics.<br>
<br>
The endeavor was driven by the idea of leveraging artificial intelligence to understand and manipulate DNA sequences. This ambitious undertaking combined elements of biology and machine learning to create a system capable of classifying DNA sequences and, more impressively, generating new, synthetic sequences.<br>
<br>
However, it's essential to note that this project represents an initial attempt and the process we've employed is not yet refined. While we've made significant strides, there's still ample scope for improvement and optimization in many aspects. The models and methods used, including artificial neural networks (ANN) for sequence classification and generative adversarial networks (GAN) for generating synthetic sequences, serve as a baseline that we expect to refine further.<br>
<br>
This project is a testament to the potential of interdisciplinary collaboration, demonstrating what can be achieved when we bridge the gap between disparate fields like biology and artificial intelligence. While we've embarked on this journey with a strong start, this is just the beginning. We invite researchers, enthusiasts, and citizen scientists alike to participate, contribute, and help improve this open-source effort.

## Methodology and Results

### The Code 

The following code accessing GenBank and reads in the files of interest which are recombinant adeno-associated virus vectors used for the treatment of SARS-CoV-2.<br>
<br>
Link to the academic paper that uses these sequences: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8927296/

#### Importing Libraries 

The script begins by importing the necessary libraries. These include numpy and pandas for handling data, BioPython for interacting with biological data, TensorFlow for machine learning, and scikit-learn for data splitting.

#### Functions for the GAN Model and Sequence Processing

Next, several functions are defined to handle specific tasks related to DNA sequences and the generative adversarial network (GAN) model. These include functions to:<br>
<br>
1) Preprocess DNA sequences into numerical representations.<br>
2) Construct the generator and discriminator parts of the GAN.<br>
3) Fetch a DNA sequence from the GenBank database.<br>
4) Introduce random mutations into a DNA sequence.<br>
5) Generate 'false positive' sequences by mutating real sequences.<br>
6) Create an Artificial Neural Network (ANN) model.

#### Main Execution and Data Preparation 

In the main execution block of the script, it starts by defining a list of GenBank accession numbers and fetches the corresponding sequences. It also generates a number of 'false positive' sequences by introducing random mutations into the real sequences. All the sequences (real and false positive) are then preprocessed and combined with their labels into a DataFrame, which is saved as a CSV file.

#### Building and Training the ANN Model 

An ANN model is built using the previously defined function and is trained using the preprocessed sequences and their labels. The model is then evaluated on a validation set and the trained model is saved.

#### Building and Training the GAN Model 

The generator and discriminator components of the GAN are built, compiled, and combined into the full GAN model. The GAN is trained in a loop, with the discriminator trained on real and generated sequences and the generator trained to try and fool the discriminator.

#### Generation of New Sequences

After training, the GAN's generator is used to generate a set of new sequences, which are then decoded back into DNA sequences and saved to a file.

#### Classification of Generated Sequences 

This part is commented out but it hints at using the previously trained ANN model to classify the newly generated sequences as 'worthy' or 'unworthy'. The sequences would need to be preprocessed before they can be fed to the ANN model.

#### Working Model Generator

In [49]:
import numpy as np
import pandas as pd
import random
from Bio import Entrez, SeqIO
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

def preprocess_sequences(sequences, max_length):
    nucleotides = "ACGT"
    num_nucleotides = len(nucleotides)
    num_sequences = len(sequences)
    encoded_sequences = np.zeros((num_sequences, max_length, num_nucleotides), dtype=np.int8)

    for i, seq in enumerate(sequences):
        for j, nt in enumerate(seq):
            if nt in nucleotides:
                index = nucleotides.index(nt)
                encoded_sequences[i, j, index] = 1
            else:
                encoded_sequences[i, j, :] = [0, 0, 0, 0]

    return encoded_sequences

def build_generator(latent_dim, max_length, num_nucleotides):
    input_noise = Input(shape=(latent_dim,))
    gen = Dense(128, activation='relu')(input_noise)
    gen = Dense(max_length * num_nucleotides, activation='sigmoid')(gen)
    output_seq = Reshape((max_length, num_nucleotides))(gen)
    
    return Model(input_noise, output_seq)

def build_discriminator(max_length, num_nucleotides):
    input_seq = Input(shape=(max_length, num_nucleotides))
    dis = Flatten()(input_seq)
    dis = Dense(128, activation='relu')(dis)
    output = Dense(1, activation='sigmoid')(dis)
    
    return Model(input_seq, output)

def build_gan(generator, discriminator):
    discriminator.trainable = False
    gan_input = Input(shape=(latent_dim,))
    generated_seq = generator(gan_input)
    gan_output = discriminator(generated_seq)
    gan = Model(gan_input, gan_output)
    
    return gan

def fetch_sequence_and_product(genbank_accession):
    Entrez.email = "biotuck@gmail.com"
    try:
        handle = Entrez.efetch(db="nucleotide", id=genbank_accession, rettype="gb", retmode="text")
        record = SeqIO.read(handle, "genbank")
        handle.close()
    except Exception as e:
        raise ValueError(f"Error fetching sequence for accession {genbank_accession}: {e}")

    sequence = str(record.seq)
    product = ""

    for feature in record.features:
        if feature.type == "CDS" and "product" in feature.qualifiers:
            product = feature.qualifiers["product"][0]
            break

    return sequence, product

def mutate_sequence(seq, num_mutations):
    nucleotides = list("ACGT")
    seq_list = list(seq)
    
    for _ in range(num_mutations):
        idx_to_change = random.randint(0, len(seq_list) - 1)
        new_nt = random.choice(nucleotides)
        seq_list[idx_to_change] = new_nt
    
    return ''.join(seq_list)
def generate_false_positives(sequences, num_fp, num_mutations):
    alse_positives = []
    
    for _ in range(num_fp):
        seq_to_mutate = random.choice(sequences)
        false_positives.append(mutate_sequence(seq_to_mutate, num_mutations))
    
    return false_positives

# New function to create ANN model
def create_model(input_dim):
    model = Sequential()
    model.add(Dense(128, input_dim=input_dim, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
    return model

if __name__ == "__main__":
    latent_dim = 100  # Define latent_dim here

    genbank_accessions = ["OK272506.1", "OK272507.1", "OK272508.1", "OK272509.1", "OK272510.1", "OK272511.1"]

    # Fetch sequences
    sequences = []
    valid_accessions = []  # Keep track of valid accessions
    for accession in genbank_accessions:
        try:
            sequence, product = fetch_sequence_and_product(accession)
            sequences.append(sequence)
            valid_accessions.append(accession)
        except ValueError as e:
            print(f"Unable to fetch sequence for {accession}: {str(e)}")

    if not sequences:
        print("Error: No valid sequences found for any accession numbers. Exiting.")
        exit(1)

    num_nucleotides = 4  # A, C, G, T

    # Generate false positives
    num_fp = 24
    num_mutations = 100
    false_positives = generate_false_positives(sequences, num_fp, num_mutations)

    # Combine the original sequences and the false positives
    all_sequences = sequences + false_positives

    # Encode the sequences
    max_length = max(len(seq) for seq in all_sequences)
    encoded_sequences = preprocess_sequences(all_sequences, max_length)

    # Generate labels for the sequences
    # The 'worthy' sequences are represented by 1 and the 'unworthy' sequences are represented by 0
    labels = [1]*len(sequences) + [0]*len(false_positives)

    # Combine the encoded sequences and the labels
    df_ann = pd.DataFrame(encoded_sequences.reshape(encoded_sequences.shape[0], -1))
    df_ann['label'] = labels

    # Save the dataframe to a CSV file
    df_ann.to_csv("df_ann.csv", index=False)

    # Split the data
    X = df_ann.iloc[:,:-1]
    y = df_ann.iloc[:,-1]
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    ## Create and train the ANN model
    ann_model = create_model(X_train.shape[1])
    history = ann_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=32)

    # Evaluate the model on the validation set
    loss, accuracy = ann_model.evaluate(X_val, y_val, verbose=0)
    print(f"Final accuracy on the validation set: {accuracy*100:.2f}%")

    # Save the trained ANN model
    ann_model.save("ann_model.h5")

    # Build GAN
    generator = build_generator(latent_dim, max_length, num_nucleotides)
    discriminator = build_discriminator(max_length, num_nucleotides)
    gan = build_gan(generator, discriminator)

    # Compile the discriminator
    discriminator.compile(loss='binary_crossentropy', optimizer=Adam())
    gan.compile(loss='binary_crossentropy', optimizer=Adam())

    # Compile the combined model
    gan.compile(loss='binary_crossentropy', optimizer=Adam())


    # Training loop
    epochs = 1000
    batch_size = 32

    for epoch in range(epochs):
        # Select a random batch of real data
        idx = np.random.randint(0, encoded_sequences.shape[0], batch_size)
        real_seqs = encoded_sequences[idx]

        # Generate a batch of new data
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        gen_seqs = generator.predict(noise)

        # Train the discriminator
        d_loss_real = discriminator.train_on_batch(real_seqs, np.ones((batch_size, 1)))
        d_loss_fake = discriminator.train_on_batch(gen_seqs, np.zeros((batch_size, 1)))
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train the generator
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))

        # If at save interval, print training losses and save generated sequence samples
        if epoch % 100 == 0:
            print(f"Epoch: {epoch} | Discriminator Loss: {d_loss} | Generator Loss: {g_loss}")

    # Save the GAN models
    generator.save("gan_generator.h5")
    discriminator.save("gan_discriminator.h5")
    gan.save("gan_model.h5")
    
    # Generate new sequences using the generator
    num_generated_seqs = 25
    noise = np.random.normal(0, 1, (num_generated_seqs, latent_dim))
    generated_seqs = generator.predict(noise)

    # Decode the generated sequences back to nucleotides
    generated_nucleotide_seqs = []
    for gen_seq in generated_seqs:
        nucleotides = "ACGT"
        generated_nucleotide_seq = ''.join([nucleotides[np.argmax(nt)] for nt in gen_seq])
        generated_nucleotide_seqs.append(generated_nucleotide_seq)

    # Print the generated sequences
    print("Generated Sequences:")
    for i, gen_seq in enumerate(generated_nucleotide_seqs, 1):
        print(f"{i}. {gen_seq}")

with open('generated_sequences.txt', 'w') as f:
    for seq in generated_nucleotide_seqs:
        f.write(f"{seq}\n")
    # At this point, you can use the trained ANN model to predict the worthiness of the generated sequences
    # You need to encode the generated sequences and reshape them appropriately before passing them to the model

Epoch 1/100
4/4 [==============================] - 1s 130ms/step - loss: 0.7394 - accuracy: 0.8167 - val_loss: 1.8079e-04 - val_accuracy: 1.0000
Epoch 2/100
4/4 [==============================] - 0s 23ms/step - loss: 0.3944 - accuracy: 0.9500 - val_loss: 0.1852 - val_accuracy: 1.0000
Epoch 3/100
4/4 [==============================] - 0s 22ms/step - loss: 0.2450 - accuracy: 0.9500 - val_loss: 0.0142 - val_accuracy: 1.0000
Epoch 4/100
4/4 [==============================] - 0s 24ms/step - loss: 0.2578 - accuracy: 0.9500 - val_loss: 0.0284 - val_accuracy: 1.0000
Epoch 5/100
4/4 [==============================] - 0s 24ms/step - loss: 0.2597 - accuracy: 0.9500 - val_loss: 0.0496 - val_accuracy: 1.0000
Epoch 6/100
4/4 [==============================] - 0s 22ms/step - loss: 0.2516 - accuracy: 0.9500 - val_loss: 0.0175 - val_accuracy: 1.0000
Epoch 7/100
4/4 [==============================] - 0s 24ms/step - loss: 0.2522 - accuracy: 0.9500 - val_loss: 0.0827 - val_accuracy: 1.0000
Epoch 8/100
4/4

4/4 [==============================] - 0s 21ms/step - loss: 0.0061 - accuracy: 1.0000 - val_loss: 0.2377 - val_accuracy: 1.0000
Epoch 60/100
4/4 [==============================] - 0s 22ms/step - loss: 0.0058 - accuracy: 1.0000 - val_loss: 0.2538 - val_accuracy: 1.0000
Epoch 61/100
4/4 [==============================] - 0s 24ms/step - loss: 0.0057 - accuracy: 1.0000 - val_loss: 0.2549 - val_accuracy: 1.0000
Epoch 62/100
4/4 [==============================] - 0s 25ms/step - loss: 0.0052 - accuracy: 1.0000 - val_loss: 0.3338 - val_accuracy: 0.8667
Epoch 63/100
4/4 [==============================] - 0s 30ms/step - loss: 0.0054 - accuracy: 1.0000 - val_loss: 0.3410 - val_accuracy: 0.8667
Epoch 64/100
4/4 [==============================] - 0s 22ms/step - loss: 0.0048 - accuracy: 1.0000 - val_loss: 0.2276 - val_accuracy: 1.0000
Epoch 65/100
4/4 [==============================] - 0s 24ms/step - loss: 0.0050 - accuracy: 1.0000 - val_loss: 0.2278 - val_accuracy: 1.0000
Epoch 66/100
4/4 [========

#### Testing the Generated Sequences and Model Performances

In [51]:
from tensorflow.keras.models import load_model
import numpy as np
import shap

# Load the model
model = load_model('ann_model.h5')

# Load the generated sequences
with open('generated_sequences.txt', 'r') as f:
    generated_sequences = [line.strip() for line in f]

# Preprocess the sequences
max_length = max(len(seq) for seq in generated_sequences)
encoded_sequences = preprocess_sequences(generated_sequences, max_length)

# Reshape the sequences for prediction
encoded_sequences = encoded_sequences.reshape(encoded_sequences.shape[0], -1)

# Make predictions
predictions = model.predict(encoded_sequences)

# Create an explainer
explainer = shap.DeepExplainer(model, encoded_sequences)

# Compute the SHAP values for each prediction
shap_values = explainer.shap_values(encoded_sequences)

# Create a mapping from encoded nucleotides to their names
nucleotide_map = {0: 'A', 1: 'C', 2: 'G', 3: 'T'}

# Evaluate the sequences
for i, (pred, shap_val) in enumerate(zip(predictions, shap_values[0])):  # Note that shap_values is a list with one element
    # Calculate total contribution of each nucleotide
    nucleotide_contributions = {nucleotide: 0 for nucleotide in nucleotide_map.values()}
    for j in range(0, len(shap_val), 4):
        for k in range(4):
            nucleotide_contributions[nucleotide_map[k]] += shap_val[j+k]
    print(f"Sequence {i+1}: {'Worthy' if pred >= 0.5 else 'Unworthy'}")
    print("Contribution of each nucleotide to the prediction:")
    for nucleotide, contribution in nucleotide_contributions.items():
        print(f"{nucleotide}: {contribution}")


Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.
`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.


Sequence 1: Unworthy
Contribution of each nucleotide to the prediction:
A: 0.0
C: 0.0
G: 0.0
T: 0.0
Sequence 2: Unworthy
Contribution of each nucleotide to the prediction:
A: 0.0
C: 0.0
G: 0.0
T: 0.0
Sequence 3: Unworthy
Contribution of each nucleotide to the prediction:
A: 0.0
C: 0.0
G: 0.0
T: 0.0
Sequence 4: Unworthy
Contribution of each nucleotide to the prediction:
A: 0.0
C: 0.0
G: 0.0
T: 0.0
Sequence 5: Unworthy
Contribution of each nucleotide to the prediction:
A: 0.0
C: 0.0
G: 0.0
T: 0.0
Sequence 6: Unworthy
Contribution of each nucleotide to the prediction:
A: 0.0
C: 0.0
G: 0.0
T: 0.0
Sequence 7: Unworthy
Contribution of each nucleotide to the prediction:
A: 0.0
C: 0.0
G: 0.0
T: 0.0
Sequence 8: Unworthy
Contribution of each nucleotide to the prediction:
A: 0.0
C: 0.0
G: 0.0
T: 0.0
Sequence 9: Unworthy
Contribution of each nucleotide to the prediction:
A: 0.0
C: 0.0
G: 0.0
T: 0.0
Sequence 10: Unworthy
Contribution of each nucleotide to the prediction:
A: 0.0
C: 0.0
G: 0.0
T: 0.0

## Conclusion 

In this project, we explored the application of an artificial neural network (ANN) and a generative adversarial network (GAN) to work with DNA sequence data. We successfully used an ANN for sequence classification and a GAN for generating synthetic DNA sequences. The ANN achieved an accuracy of 74% on our validation set, which is a fair result but suggests room for improvement.<br>
<br>
There might have been flaws in the methodology that we employed to train our ANN. For instance, our choice of input sequences may have contributed to the model's suboptimal performance. In this study, we assumed that all sequences sourced from our dataset were effective sequences. This assumption may have introduced noise into the training data, potentially reducing the model's ability to accurately distinguish between 'worthy' and 'unworthy' sequences. <br>
<br>
In the context of the GAN, while the model was able to generate synthetic sequences, the quality of these sequences could potentially be improved. It's clear that the parameters of the GAN require further optimization. This could involve tuning the architecture of the generator and discriminator networks, adjusting the training hyperparameters, or experimenting with different methods of training the GAN.<br>
<br>
Despite these challenges, this project provided an interesting demonstration of how a GAN can be applied to sequence data to generate synthetic information. It showcased the potential of these models in the field of genomics, offering a novel approach to generating new sequences for study.<br>
<br>
Moving forward, we plan to refine our methods and models to improve performance. This includes reassessing our assumptions about the training data for the ANN and optimizing the parameters of the GAN. Through these improvements, we aim to enhance the model's ability to generate high-quality, synthetic DNA sequences and more accurately classify them.